In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import os
from datetime import datetime, timedelta

C:\Users\rando\AppData\Roaming\Python\Python39\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# for colab environment
prefix  = 'https://drive.google.com/uc?export=download&id='
import_data = pd.read_csv(prefix + 'https://drive.google.com/file/d/16pfQT9f5s7gc5_0pBJNEcevCBhxftSMt/view?usp=sharing'.split('/')[-2])
export_data = pd.read_csv(prefix + 'https://drive.google.com/file/d/1m750gh4aqzyzsSIBnI6wbkKzw3XqE5qe/view?usp=sharing'.split('/')[-2])
net_import_data = pd.read_csv(prefix + 'https://drive.google.com/file/d/14Y-VX9qNnLvlvsfTJOdVctowhvLcIk6B/view?usp=sharing'.split('/')[-2])
price = pd.read_csv(prefix + 'https://drive.google.com/file/d/1If41nY3li59EzG0H147w4_Uj1KQySk0T/view?usp=sharing'.split('/')[-2])

# for local environment
# import_data = pd.read_csv('PET_MOVE_WKLY_DC_NUS-Z00_MBBLPD_W_1.csv')
# export_data = pd.read_csv('PET_MOVE_WKLY_DC_NUS-Z00_MBBLPD_W_2.csv')
# net_import_data = pd.read_csv('PET_MOVE_WKLY_DC_NUS-Z00_MBBLPD_W_3.csv')
# price = pd.read_csv('EMM_EPM0_PTE_NUS_DPGw.csv')

In [3]:
import_data.columns = import_data.iloc[1, :]
import_data = import_data.iloc[2:, :]

export_data.columns = export_data.iloc[1, :]
export_data = export_data.iloc[2:, :]

net_import_data.columns = net_import_data.iloc[1, :]
net_import_data = net_import_data.iloc[2:, :]

price.columns = price.iloc[1, :]
price = price.iloc[2:-1, :]

In [4]:
price['Date'] = pd.to_datetime(price['Date'], format='%b %d, %Y')
import_data['Date'] = pd.to_datetime(import_data['Date'], format='%b %d, %Y')
export_data['Date'] = pd.to_datetime(export_data['Date'], format='%b %d, %Y')
net_import_data['Date'] = pd.to_datetime(net_import_data['Date'], format='%b %d, %Y')

In [5]:
price['Date'] = price['Date'] - timedelta(days=3)

In [6]:
data = (price.merge(import_data, how='left', on='Date')
    .merge(export_data, how='left', on='Date')
    .merge(net_import_data, how='left', on='Date')
)

data = data.dropna(how='any', axis=1)
data.head()

1,Date,Weekly U.S. All Grades All Formulations Retail Gasoline Prices (Dollars per Gallon),Weekly U.S. Imports of Crude Oil and Petroleum Products (Thousand Barrels per Day),Weekly U.S. Imports of Crude Oil (Thousand Barrels per Day),Weekly U.S. Commercial Crude Oil Imports Excluding SPR (Thousand Barrels per Day),Weekly U.S. Crude Oil Imports by SPR (Thousand Barrels per Day),Weekly U.S. Imports of Total Petroleum Products (Thousand Barrels per Day),Weekly U.S. Imports of Total Gasoline (Thousand Barrels per Day),Weekly U.S. Imports of Gasoline Blending Components (Thousand Barrels per Day),Weekly U.S. Imports of Kerosene-Type Jet Fuel (Thousand Barrels per Day),Weekly U.S. Imports of Distillate Fuel Oil (Thousand Barrels per Day),Weekly U.S. Imports of Residual Fuel Oil (Thousand Barrels per Day),Weekly U.S. Exports of Crude Oil and Petroleum Products (Thousand Barrels per Day),Weekly U.S. Exports of Crude Oil (Thousand Barrels per Day),Weekly U.S. Net Imports of Crude Oil and Petroleum Products (Thousand Barrels per Day)
0,1993-04-02,1.068,7501,5512,5512,0,1989,273,26,121,265,456,986,108,6515
1,1993-04-09,1.079,8956,7369,7243,126,1587,328,74,53,260,223,986,108,7970
2,1993-04-16,1.079,7976,6406,6406,0,1570,343,19,100,280,235,986,108,6990
3,1993-04-23,1.086,8715,6804,6662,142,1911,396,43,71,175,320,986,108,7729
4,1993-04-30,1.086,9418,7731,7518,213,1687,209,56,90,120,516,981,111,8437


In [7]:
data.columns

Index(['Date',
       'Weekly U.S. All Grades All Formulations Retail Gasoline Prices  (Dollars per Gallon)',
       'Weekly U.S. Imports of Crude Oil and Petroleum Products  (Thousand Barrels per Day)',
       'Weekly U.S. Imports of Crude Oil  (Thousand Barrels per Day)',
       'Weekly U.S. Commercial Crude Oil Imports Excluding SPR  (Thousand Barrels per Day)',
       'Weekly U.S. Crude Oil Imports by SPR  (Thousand Barrels per Day)',
       'Weekly U.S. Imports of Total Petroleum Products  (Thousand Barrels per Day)',
       'Weekly U.S. Imports of Total Gasoline  (Thousand Barrels per Day)',
       'Weekly U.S. Imports of Gasoline Blending Components  (Thousand Barrels per Day)',
       'Weekly U.S. Imports of Kerosene-Type Jet Fuel  (Thousand Barrels per Day)',
       'Weekly U.S. Imports of Distillate Fuel Oil  (Thousand Barrels per Day)',
       'Weekly U.S. Imports of Residual Fuel Oil  (Thousand Barrels per Day)',
       'Weekly U.S. Exports of Crude Oil and Petroleum Produc

In [8]:
df = data.sort_values(by='Date')
df['Date'] = df['Date'].values.astype(np.int64) // 10 ** 9
df.head()

1,Date,Weekly U.S. All Grades All Formulations Retail Gasoline Prices (Dollars per Gallon),Weekly U.S. Imports of Crude Oil and Petroleum Products (Thousand Barrels per Day),Weekly U.S. Imports of Crude Oil (Thousand Barrels per Day),Weekly U.S. Commercial Crude Oil Imports Excluding SPR (Thousand Barrels per Day),Weekly U.S. Crude Oil Imports by SPR (Thousand Barrels per Day),Weekly U.S. Imports of Total Petroleum Products (Thousand Barrels per Day),Weekly U.S. Imports of Total Gasoline (Thousand Barrels per Day),Weekly U.S. Imports of Gasoline Blending Components (Thousand Barrels per Day),Weekly U.S. Imports of Kerosene-Type Jet Fuel (Thousand Barrels per Day),Weekly U.S. Imports of Distillate Fuel Oil (Thousand Barrels per Day),Weekly U.S. Imports of Residual Fuel Oil (Thousand Barrels per Day),Weekly U.S. Exports of Crude Oil and Petroleum Products (Thousand Barrels per Day),Weekly U.S. Exports of Crude Oil (Thousand Barrels per Day),Weekly U.S. Net Imports of Crude Oil and Petroleum Products (Thousand Barrels per Day)
0,733708800,1.068,7501,5512,5512,0,1989,273,26,121,265,456,986,108,6515
1,734313600,1.079,8956,7369,7243,126,1587,328,74,53,260,223,986,108,7970
2,734918400,1.079,7976,6406,6406,0,1570,343,19,100,280,235,986,108,6990
3,735523200,1.086,8715,6804,6662,142,1911,396,43,71,175,320,986,108,7729
4,736128000,1.086,9418,7731,7518,213,1687,209,56,90,120,516,981,111,8437


In [9]:
df.to_pickle('working_dataset.pkl')

In [10]:
test_dataset = df.reset_index(drop=True).iloc[-int(len(df)*0.3):, :]
test_dataset.to_pickle('test_dataset.pkl')